# Convolutional networks

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import sys

from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
import numpy as np

## Convolutional layer
In convolutional networks processing is localized spatially and behaviour replicates across space. This is expressed as a convolution. In their basic form, convolutional units are described by size and stride.

In [2]:
# a few activation functions to play with
def halfSquare(x):
    return tf.square(tf.nn.relu(x))

def myIdty(x):
    return x

act_list = {'sigmoid': tf.nn.sigmoid,     \
            'tanh': tf.nn.tanh,        \
            'relu': tf.nn.relu,        \
            'abs': tf.abs,         \
            'square': tf.square,      \
            'half_square': halfSquare, \
            'identity': myIdty}

def initWeightBias(shape, stddev=0.1, bias=0.1): 
    initW = tf.truncated_normal(shape, stddev=stddev)
    initBias = tf.constant(bias, shape=[shape[-1]])
    return initW, initBias

class ConvLayer(object):
    def __init__(self, shape, stride, activation, padding='SAME'):
        self.shape = shape
        self.stride = stride
        self.padding = padding
        initW, initBias = initWeightBias(self.shape)
        self.params = {}
        self.params['W'] = tf.Variable(initW)
        self.params['b'] = tf.Variable(initBias)
        self.g = act_list[activation]
    
    def forward(self, X):
        Z = tf.nn.conv2d(X, self.params['W'], strides=[1, self.stride, self.stride, 1], padding=self.padding) + self.params['b']
        return self.g(Z)


## Pooling Layer

In [3]:
pool_type = {'MAX':tf.nn.max_pool, \
             'AVG':tf.nn.avg_pool}
class PoolLayer(object):
    def __init__(self, shape, stride, pooling='AVG', padding='SAME'):
        self.shape = (1, shape[0], shape[1], 1)
        self.stride = (1, stride, stride, 1)
        self.pooltype = pooling
        self.padding = padding
        self.pool = pool_type[self.pooltype]
    
    def forward(self, X):
        return self.pool(X, self.shape, self.stride, self.padding)

## Fully Connected Layer

In [4]:
class FullyConnectedLayer(object):
    def __init__(self, shape, activation):
        self.shape = shape
        initW, initBias = initWeightBias(self.shape)
        self.params = {}
        self.params['W'] = tf.Variable(initW)
        self.params['b'] = tf.Variable(initBias)
        self.g = act_list[activation]
    
    def forward(self, X):
        Z = tf.matmul(X, self.params['W']) + self.params['b']
        return self.g(Z)

## Convolutional Network

In [5]:
class MnistConvNet():
    def __init__(self):
        self.conv1 = ConvLayer((5, 5, 1, 6), 1,'relu')
        self.pool1 = PoolLayer((2, 2), 2,'MAX', 'VALID')
        self.conv2 = ConvLayer((5, 5, 6, 16), 1,'relu', 'VALID')
        self.pool2 = PoolLayer((2, 2), 2,'MAX', 'VALID')
        self.conv3 = ConvLayer((5, 5, 16, 120), 1,'relu', 'VALID')
        self.fc4 = FullyConnectedLayer((120, 84), 'relu')
        self.fc5 = FullyConnectedLayer((84, 10), 'identity')
        
    def forward(self, X):
        conv1 = self.conv1.forward(X)
        pool1 = self.pool1.forward(conv1)
        conv2 = self.conv2.forward(pool1)
        pool2 = self.pool2.forward(conv2)
        conv3 = self.conv3.forward(pool2)
        fc4 = self.fc4.forward(tf.reshape(conv3, [-1, 120]))
        fc5 = self.fc5.forward(fc4)
        return fc5
my_net = MnistConvNet()

## Train the CNN

In [6]:
# Data handler included in TF examples (downloads data)
mnist = input_data.read_data_sets('../data/mnist', one_hot=True)
x = tf.placeholder(tf.float32, [None, 28, 28, 1])
pred = my_net.forward(x)
# create loss to train
label = tf.placeholder(tf.float32, shape=[None, 10])
# Use combined logsig and xent to have numerically stable gradients
xent_cost = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(labels=label, logits=pred))
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(xent_cost)

sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

# Train
correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(label, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


for i in range(10000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    ## display training accuracy 
    if i%1000 == 0:
        train_accuracy = accuracy.eval(feed_dict={
                x:np.reshape(batch_xs, [-1, 28, 28, 1]), label: batch_ys})
        train_cost = xent_cost.eval(feed_dict={
                x:np.reshape(batch_xs, [-1, 28, 28,1]), label: batch_ys})
        print("step %d, training cost %g, training accuracy %g"%(i, train_cost, train_accuracy))
    ## 
    sess.run(train_step, feed_dict={x: np.reshape(batch_xs, [-1, 28, 28,1]), label: batch_ys})

Extracting ../data/mnist/train-images-idx3-ubyte.gz
Extracting ../data/mnist/train-labels-idx1-ubyte.gz
Extracting ../data/mnist/t10k-images-idx3-ubyte.gz
Extracting ../data/mnist/t10k-labels-idx1-ubyte.gz
step 0, training cost 2.31126, training accuracy 0.07
step 1000, training cost 0.211454, training accuracy 0.95
step 2000, training cost 0.326092, training accuracy 0.89
step 3000, training cost 0.103977, training accuracy 0.97
step 4000, training cost 0.190011, training accuracy 0.95
step 5000, training cost 0.0799839, training accuracy 0.98
step 6000, training cost 0.151034, training accuracy 0.96
step 7000, training cost 0.049047, training accuracy 0.98
step 8000, training cost 0.0468301, training accuracy 0.98
step 9000, training cost 0.113455, training accuracy 0.96
